In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models  # torchvision package contains many types of datasets (including MNIST dataset) and pre-trained models
from torchvision.utils import save_image

import numpy as np
import matplotlib.pyplot as plt
import math

import os
import random
import time
from datetime import timedelta, datetime
from PIL import Image
import cv2  # for video
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# kaggle API
# !pip install -q kaggle
# upload kaggle.json
# files.upload()

# create a kaggle folder
# ! mkdir ~/.kaggle
# copy the kaggle.json to folder created
# ! cp kaggle.json ~/.kaggle/
# permission for the json to act
# ! chmod 600 ~/.kaggle/kaggle.json

# to list all datasets in kaggle
# ! kaggle datasets list
# Van Gogh Paintings
# ! unzip .zip

******************************************************************************

Supporting_Functions

******************************************************************************



In [ ]:
def load_image(path, max_size=600, shape=None):
        image = Image.open(path).convert('RGB')
        size = max(image.size)

        if size > max_size:
            size = max_size

        if shape is not None:
            size = shape

        # transform image to be compatible with the model
        transform = transforms.Compose([
            transforms.Resize(size),
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,))])

        image = transform(image).unsqueeze(0)  # to add extra dimensionality
        return image

def select_image_from_dataset(dir_artists, dir_van_gogh):
      # choice = input("Van Gogh(V) or All Artists(A):")
      choice = 'A'
      if choice == 'V':
        for i, filename in enumerate(os.listdir(dir_van_gogh)):
          print(filename)
        artist = input("Select type:")
        source_folder = os.path.join(dir_van_gogh, artist)
        image = random.choice(os.listdir(source_folder))        
        while not (image.endswith(".jpg") or image.endswith(".jpeg")):
          image = random.choice(os.listdir(source_folder))
        print('selected image name:', image)
        return os.path.join(source_folder, image)
      else:
        for i, filename in enumerate(os.listdir(dir_artists)):
          print(filename)
        # artist = input("Select artist:")
        artist = 'Afremov'
        source_folder = os.path.join(dir_artists, artist)
        image = random.choice(os.listdir(source_folder))        
        while not (image.endswith(".jpg") or image.endswith(".jpeg")):
          image = random.choice(os.listdir(source_folder))
        print('selected image name:', image)
        return os.path.join(source_folder, image)

def plot_images_bunch(images, names, nrows=1):
      
      fig, axeslist = plt.subplots(ncols=int(math.ceil(len(images)/nrows)), nrows=nrows)
      for i in range(len(images)):
        axeslist.ravel()[i].imshow(SF.image_convert_to_numpy(images[i]))
        axeslist.ravel()[i].set_title(names[i])
        axeslist.ravel()[i].set_axis_off()
      plt.tight_layout()
      if len(images) > 2:
        plt.savefig('all_images.jpg', dpi=500, bbox_inches='tight')
      plt.show()

# this class includes some general supporting functions for CNN Transfer Learning
class Supporting_Functions:
    def __init__(self, log_filename, content_image_path, style_image_path, device, content_weight, style_weight, learning_rate, steps, storage_limit):
        super().__init__()
        self.log_filename = log_filename
        self.content_image_path = content_image_path
        self.style_image_path = style_image_path
        self.device = device
        self.content_weight = content_weight
        self.style_weight = style_weight
        self.learning_rate = learning_rate
        self.steps = steps
        self.storage_limit = storage_limit

    def enter_log(self, text, header=False):
        file = open(self.log_filename, "a")

        if header:
            log_str = str(datetime.now().strftime("%d/%m/%Y %H:%M:%S")) + "\nContent Image: " + self.content_image_path + "\nStyle Image: " + self.style_image_path \
                      + "\ndevice: " + str(self.device) + ", content_weight: " + str(self.content_weight) + ", style_weight: " + str(self.style_weight) + ", learning_rate: " \
                      + str(self.learning_rate) + ", steps: " + str(self.steps) + ", storage_limit: " + str(self.storage_limit)
            file.write("\n\n" + log_str + "\n\n")

        file.write(text + "\n")
        file.close()

    # 1st dimension: color, 2nd dimension: width, 3rd dimension: height of image and pixels
    def image_convert_to_numpy(self, tensor):
        image = tensor.clone().detach().cpu().numpy()  # clones to tensor and transforms to numpy array. OR tensor.cpu().clone().detach().numpy()
        image = image.squeeze()
        image = image.transpose(1, 2, 0)
        # print(image.shape)                                                                            # (28, 28, 1)
        # denormalize image
        image = image * np.array((0.5,)) + np.array((0.5,))
        image = image.clip(0, 1)
        return image

    def create_video(self, images, name):
      frame_per_sec = 30
      frame_height, frame_width, _ = images[0].shape
      video = cv2.VideoWriter(name + '.avi', cv2.VideoWriter_fourcc(*'DIVX'), frame_per_sec, (frame_width, frame_height))

      for i in range(len(images)):
        cur_image = images[i]
        # make current image RGB
        cur_image = cur_image * 255
        cur_image = np.array(cur_image, dtype=np.uint8)
        cur_image = cv2.cvtColor(cur_image, cv2.COLOR_BGR2RGB)
        video.write(cur_image)

      video.release()
      # video is fully written and ready to be saved
      # cv2.destroyAllWindows()
      SF.enter_log('Video created.')

    def plot_images(self, images, nrows=1):
      names = ['Content Image', 'Style Image', 'Target Image']
      fig, axeslist = plt.subplots(ncols=int(len(images)/nrows), nrows=nrows)
      for i in range(len(images)):
        axeslist.ravel()[i].imshow(SF.image_convert_to_numpy(images[i]))
        axeslist.ravel()[i].set_title(names[i])
        axeslist.ravel()[i].set_axis_off()
      plt.tight_layout()
      if len(images) > 2:
        plt.savefig('three_images.jpg', dpi=500, bbox_inches='tight')
      plt.show()

    def save_image(self, image, name):
      # save_image(image, fp=name, normalize=True)
      plt.imshow(SF.image_convert_to_numpy(image))
      plt.axis('off')
      plt.savefig(name, dpi=500, bbox_inches='tight')
      # plt.show()

    
      
      


******************************************************************************

Training Algorithm

******************************************************************************


In [ ]:
# HYPERPARAMETERS
# assign weight to each style layer for representation power (early layers have more style)
style_weights = {'conv1_1': 1.0,
                 'conv2_1': 0.78,
                 'conv3_1': 0.25,
                 'conv4_1': 0.22,
                 'conv5_1': 0.18}

# dictionary that holds the specific layer numbers where features will be extracted. You can play with them.
# Conv1_1, Conv2_1, Conv3_1, Conv4_1, Conv4_2, Conv5_1
layers = {'0': 'conv1_1',    # style extraction
          '5': 'conv2_1',    # style extraction
          '10': 'conv3_1',   # style extraction
          '19': 'conv4_1',   # style extraction
          '21': 'conv4_2',   # content extraction
          '28': 'conv5_1'}   # style extraction

content_layer = 'conv4_2'


# AUXILARY PARAMETERS
# step number
storage_limit = 300

# print variables
print_per = 10
show_per = 50


# datasets
dataset_directory_artists = '/content/drive/MyDrive/CS-464 Project/CNN_files/Artists'
dataset_directory_van_gogh = '/content/drive/MyDrive/CS-464 Project/CNN_files/Van_Gogh'

content_image_path = '/content/drive/MyDrive/CS-464 Project/CNN_files/HayleyWilliams.jpg'
# style_image_path = '/content/drive/MyDrive/CS-464 Project/CNN_files/AngelOfLove.jpeg'
log_filename = "/content/drive/MyDrive/CS-464 Project/CNN_files/log_CNN_Style_Transfer.txt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # specifies run device for more optimum runtime

########################################################################################################################

# returns pre-trained VGG19 model
def get_model():
    # VGG 19 pre-trained model
    model = models.vgg19(pretrained=True).features
    # SF.enter_log('VGG 19 pre-trained model is created.')

    # freeze parameters
    for param in model.parameters():
        param.requires_grad = False

    # send model to GPU
    model.to(device=device)
    return model


# extracts the features from image using model
def get_features(image, model):
    # dict that will store the extracted features
    features = {}
    # iterate through all layers and store the on es in the layers dict
    for name, layer in model._modules.items():
        # run image through all layers
        image = layer(image)
        if name in layers:
            features[layers[name]] = image
    return features


# Gram Matrix = V(T)*V  T: Transpose
def gram_matrix(tensor):
    # takes 4D image tensor
    # reshape the tensor
    _, d, h, w = tensor.size()
    tensor = tensor.view(d, h*w)
    gram = torch.mm(tensor, tensor.t())
    return gram


def train(model, content_image, style_image, steps, content_weight, style_weight, learning_rate):
    # get content and style features
    content_features = get_features(content_image, model)
    style_features = get_features(style_image, model)

    # style features need one more step to be more useful (Gram Matrix)
    # applying Gram Matrix eliminates the remaining content information from style features
    # Gram Matrix = V(T)*V  T: Transpose
    style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

    # define target image
    target_image = content_image.clone().requires_grad_(True).to(device=device)

    # training process images
    height, width, channels = SF.image_convert_to_numpy(target_image).shape
    images = np.empty(shape=(storage_limit, height, width, channels))

    # Adam Optimizer
    optimizer = torch.optim.Adam([target_image], lr=learning_rate)

    # training process
    SF.enter_log('Training begins...')
    iter = 0
    start_training_time = time.time()
    for ii in range(1, steps + 1):
        target_features = get_features(target_image, model)
        # calculate the content loss between content and target images using Mean Squared Error
        content_loss = torch.mean((target_features[content_layer] - content_features[content_layer]) ** 2)

        # calculate style loss iterating through 5 style layers
        style_loss = 0
        for style_layer in style_weights:
            # calculate target gram for cur layer
            target_feature = target_features[style_layer]
            target_gram = gram_matrix(target_feature)
            # get corresponding style gram from the precalculated list
            style_gram = style_grams[style_layer]
            current_style_loss = style_weights[style_layer] * torch.mean((target_gram - style_gram) ** 2)
            # normalize current_style_loss
            _, d, h, w = target_feature.shape
            style_loss += current_style_loss / (d * h * w)

        # optimizer will be used to optimize the parameters of the target image according to content and style losses
        # Style Aim: is to match the target gram matrix to the style gram matrix
        # Content Aim: is to match the target features (filtered image) to the content features
        total_loss = content_weight * content_loss + style_weight * style_loss

        optimizer.zero_grad()  # reset optimizer
        total_loss.backward()
        optimizer.step()

        # data visualization throughout the training process
        # print period
        if ii % print_per == 0:
            finish_training_time = time.time()
            print_str = 'iteration: ' + str(ii) + ' loss: ' + str(total_loss.item() / (height * width)) + ' time passed: ' + str(timedelta(seconds=finish_training_time - start_training_time))
            print(print_str)
            SF.enter_log(print_str)

        # show image period
        if ii % show_per == 0:
            plt.imshow(SF.image_convert_to_numpy(target_image))
            plt.axis('off')
            plt.show()

        # store mid images period
        if ii % (steps / storage_limit) == 0:
            images[iter] = SF.image_convert_to_numpy(target_image)
            iter += 1

    SF.enter_log('Training completed.')
    return target_image, images



******************************************************************************

Main Function

******************************************************************************

In [ ]:
# HYPERPARAMETERS
# style content ratio: alpha represents content image weight and beta represents style image weight
content_weight = 1
style_weight_ = 1e6      # (1e1, 1e6)

# learning rate
learning_rate = 0.004

# steps, iterations
steps = 2100
# additional_steps = 100

style_weights_ = [1e6, 1e4, 1e2, 1e1]

# MAIN

# get content and style images
content_image = load_image(content_image_path).to(device=device)
style_image_path = select_image_from_dataset(dataset_directory_artists, dataset_directory_van_gogh)
style_image = load_image(style_image_path, shape=content_image.shape[-2:]).to(device=device)

# get VGG19 pre-trained model
model = get_model()

target_images = []
names = ['Content Image', 'Style Image', 'Target with sw: 1e6', 'Target with sw: 1e4', 'Target with sw: 1e2', 'Target with sw: 1e1']

for style_weight in style_weights_:
  print('Training is started for', style_weight)
  SF = Supporting_Functions(log_filename, content_image_path, '/from dataset', device, content_weight, style_weight, learning_rate, steps, storage_limit)
  SF.enter_log('Content and Style images are retrieved.', header=True)

  # plot content and style images
  SF.plot_images([content_image, style_image])

  # train target image
  target_image, images = train(model, content_image, style_image, steps, content_weight, style_weight, learning_rate)
  target_images.append(target_image)

  # plot content, style, and target images
  SF.plot_images([content_image, style_image, target_image])

  # save target image
  SF.save_image(target_image, 'target_image' + str(style_weight) + '.jpg')

  # create video
  SF.create_video(images, 'transformation_with_' + str(style_weight))

plot_images_bunch(np.concatenate(([content_image, style_image], target_images)), names, nrows=2)